First load the 4 files one by one (not all at once)!

In [2]:
!pip install tensorflow-addons

     |████████████████████████████████| 686kB 11.2MB/s 


In [5]:
import tensorflow as tf
import tensorflow_addons as tfa
# from tensorflow.keras.losses import Loss

from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
import numpy as np
import math
import timeit
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, UpSampling2D, Reshape, Lambda
from keras.constraints import max_norm
from PIL import Image
import keras.backend as K
import time
from random import shuffle

In [6]:
path = '/content/'

In [8]:
X_val = np.load(path + "ideal_X_val.npy")
y_val = np.load(path + "ideal_y_val.npy")

X_train = np.load(path + "ideal_X_train.npy") # [sigma, alpha, Xi, width, space, noise, LER_L, LER_R]
y_train = np.load(path + "ideal_y_train.npy")

Set quantile

In [ ]:
quantile = 0.05 # lower quantile for 90% confidence

In [ ]:
model_name = 'ideal_lower_90.h5'

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(7,))) # kernel_regularizer=l2(0.01), 
model.add(Dense(32, activation='relu')) 
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='relu'))

# try residual network

model.summary()

adam = keras.optimizers.Adam(lr=1e-3)

model.compile(loss = tfa.losses.PinballLoss(tau=quantile), # MyPinballLoss(quantile), # lambda y, y_hat: pinball_loss(y, y_hat, quantile),
              optimizer=adam)

callbacks = [
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=9, min_lr=0.0001),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=18,
        ),
    # keras.callbacks.ModelCheckpoint(
        # path + 'models/' + 'Weights/compressedweights.{epoch:02d}-{val_loss:.2f}.hdf5',
        # monitor='val_loss',
        # save_best_only=True,
        # verbose=1),
]
			  
history = model.fit(X_train[:, :7], y_train[:, 0],
             batch_size=16,
             epochs=120,
             validation_data=(X_val[:, :7], y_val[:, 0]),
             shuffle=True,
			 callbacks=callbacks)
			  
model.save(path + model_name)